In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED) 

In [ ]:
cwd = os.getcwd()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!unzip /content/drive/My\ Drive/Development_Dataset.zip

In [ ]:
!ls /content/Development_Dataset/

LICENSE.txt  Test_Dev  Training


In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

input_dir = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if input_dir:
  inp_img_data_gen = ImageDataGenerator(validation_split =0.2)
  inp_mask_data_gen = ImageDataGenerator(validation_split =0.2,)

In [ ]:
apply_data_augmentation = True
if apply_data_augmentation:
    img_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')
    mask_data_gen = ImageDataGenerator(rotation_range=10,
                                       width_shift_range=10,
                                       height_shift_range=10,
                                       zoom_range=0.3,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='reflect')
    
test_img_data_gen = ImageDataGenerator()

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'Development_Dataset')

# img

# Batch size
bs = 16

# img shape
img_h = 256
img_w = 256


# Training
img_training_dir = os.path.join(dataset_dir, 'Training/Bipbip/Haricot/')
train_img_gen = inp_img_data_gen.flow_from_directory(img_training_dir,
                                                       classes = ['Images'],
                                               target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       class_mode=None, 
                                                     interpolation='bilinear',
                                                       shuffle=True,
                                                       
                                                       seed=SEED,
                                                       subset='training')  # targets are directly converted into one-hot vectors
mask_training_dir = os.path.join(dataset_dir, 'Training/Bipbip/Haricot/')
train_mask_gen = inp_mask_data_gen.flow_from_directory(mask_training_dir,
                                               classes = ['Masks'],
                                               batch_size=bs, 
                                               class_mode=None,
                                               interpolation='bilinear',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')  # targets are directly converted into one-hot vectors
train_gen = zip(train_img_gen, train_mask_gen)

valid_img_gen = inp_img_data_gen.flow_from_directory(img_training_dir,
                                                  classes = ['Images'],
                                                  target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       class_mode=None, 
                                                     interpolation='bilinear',
                                                       shuffle=False,
                                                       
                                                       seed=SEED,
                                                  subset='validation')
valid_mask_gen = inp_mask_data_gen.flow_from_directory(mask_training_dir,
                                                  classes = ['Masks'],
                                                  target_size=(img_h, img_w),
                                                  interpolation='bilinear',
                                                       batch_size=bs,
                                                       class_mode=None, 
                                                       shuffle=False,
                                                      
                                                       seed=SEED,
                                                  subset='validation')
valid_gen = zip(valid_img_gen, valid_mask_gen)

In [ ]:
from PIL import Image

class CustomDataset(tf.keras.utils.Sequence):

  """
    CustomDataset inheriting from tf.keras.utils.Sequence.

    3 main methods:
      - __init__: save dataset params like directory, filenames..
      - __len__: return the total number of samples in the dataset
      - __getitem__: return a sample from the dataset

    Note: 
      - the custom dataset return a single sample from the dataset. Then, we use 
        a tf.data.Dataset object to group samples into batches.
      - in this case we have a different structure of the dataset in memory. 
        We have all the images in the same folder and the training and validation splits
        are defined in text files.

  """

  def __init__(self, dataset_dir, which_subset, gen, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[256, 256]):
    
    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape
    self.gen = gen

  def __len__(self):
    return len(self.gen.filenames)

  def __getitem__(self, index):
    # Read Image
    c = self.gen.filenames[index]
    c1 = c.split('/')
    c2 = c1[1].split('.')
    curr_filename = c2[0]
    if self.which_subset == 'training':
      img = Image.open(os.path.join(self.dataset_dir, 'Training/Bipbip/Haricot/Images', curr_filename + '.jpg'))
      mask = Image.open(os.path.join(self.dataset_dir, 'Training/Bipbip/Haricot/Masks', curr_filename + '.png'))
    else:
      img = Image.open(os.path.join(self.dataset_dir, 'Training/Bipbip/Haricot/Images', curr_filename + '.jpg'))
      mask = Image.open(os.path.join(self.dataset_dir, 'Training/Bipbip/Haricot/Masks', curr_filename + '.png'))

   # Resize image and mask
    img = img.resize(self.out_shape)
    mask = mask.resize(self.out_shape, resample=Image.NEAREST)
    
    img_arr = np.array(img)
    mask_arr = np.array(mask)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)
    new_mask_arr[np.where(np.all(mask_arr == [254, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [0, 0, 0], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2
    
    new_mask_arr = np.expand_dims(new_mask_arr, -1)


    if self.which_subset == 'training':
      if self.img_generator is not None and self.mask_generator is not None:
        # Perform data augmentation
        # We can get a random transformation from the ImageDataGenerator using get_random_transform
        # and we can apply it to the image using apply_transform
        img_t = self.img_generator.get_random_transform(img_arr.shape, seed=SEED)
        mask_t = self.mask_generator.get_random_transform(new_mask_arr.shape, seed=SEED)
        img_arr = self.img_generator.apply_transform(img_arr, img_t)

        out_mask = np.zeros_like(new_mask_arr)
        for c in np.unique(new_mask_arr):
          if c > 0:
            curr_class_arr = np.float32(new_mask_arr == c)
            curr_class_arr = self.mask_generator.apply_transform(curr_class_arr, mask_t)
            # from [0, 1] to {0, 1}
            curr_class_arr = np.uint8(curr_class_arr)
            # recover original class
            curr_class_arr = curr_class_arr * c 
            out_mask += curr_class_arr
    else:
      out_mask = new_mask_arr
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(out_mask)

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input 

img_h = 256
img_w = 256

dataset = CustomDataset('/content/Development_Dataset/', 'training', gen = train_img_gen,
                        img_generator=img_data_gen, mask_generator=mask_data_gen,
                        preprocessing_function=preprocess_input)
dataset_valid = CustomDataset('/content/Development_Dataset', 'validation', gen = valid_img_gen,
                              preprocessing_function=preprocess_input)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))


train_dataset = train_dataset.batch(32)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))

valid_dataset = valid_dataset.batch(32)

valid_dataset = valid_dataset.repeat()

In [ ]:
# Let's test data generator
# -------------------------
import time
from matplotlib import cm
import matplotlib.pyplot as plt

%matplotlib inline


iterator = iter(train_dataset)

In [ ]:
fig, ax = plt.subplots(1, 2)

augmented_img, target = next(iterator)
augmented_img = augmented_img[0]   # First element
augmented_img = augmented_img  # denormalize

target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)

print(np.unique(target))

target_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = [0, 0, 0]
target_img[np.where(target == 1)] = [255, 255, 255]
target_img[np.where(target == 2)] = [0, 255, 0]
# target_img[np.where(target == 3)] = [255, 0, 0]

ax[0].imshow(np.uint8(augmented_img))
ax[1].imshow(np.uint8(target_img))

plt.show()

In [ ]:
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,2): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [ ]:
def create_model(img_size,num_classes):
    inputs = tf.keras.Input(shape=img_size +(3,))
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = tf.keras.Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics=[meanIoU])
    return model


In [ ]:
img_size=(256,256)
model = create_model(img_size,num_classes=3)

# Visualize created model as a table
model.summary()

# Visualize initialized weights
# model.weights

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# ------------------
metrics = ['accuracy', meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
import os
from datetime import datetime 
callbacks = []

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


In [ ]:
model.fit(x=train_dataset,
          epochs=3,  #### set repeat in training dataset
          steps_per_epoch=len(dataset),
          validation_data=valid_dataset,
          validation_steps=len(dataset_valid), 
          callbacks=callbacks)

In [ ]:
#Preparing submission.json file
from keras.preprocessing.image import save_img
def submit(submission_dict,team,crop):
    path_mask='Development_Dataset/Test_Dev/Bipbip/Haricot/Masks'
    if not os.path.exists(path_mask):
      os.mkdir(path_mask)
    path ="Development_Dataset/Test_Dev/"
    dir= os.path.join(path,team,crop,'Images')
    predicted_images =next(os.walk(dir))[2]
    print (type(predicted_images))
    for img in predicted_images:
      mask_img = img.split(".")
      mask_img=mask_img[0]
      test_image = Image.open(os.path.join(dir,img))

      size=(256,256)
      test_image = test_image.resize(size)
      tf_test_image = tf.convert_to_tensor(
      np.array(test_image), dtype='float32'
      )
      out_sigmoid = model.predict(x=tf.expand_dims(tf_test_image, 0))
      predicted_class = tf.argmax(out_sigmoid, -1)
      predicted_class = predicted_class[0, ...]

      prediction_img = np.zeros([tf_test_image.shape[0], tf_test_image.shape[1], 3])
      prediction_img[np.where(predicted_class == 0)] = [0, 0, 0]
      prediction_img[np.where(predicted_class == 1)] = [255, 255, 255]
      prediction_img[np.where(predicted_class == 2)] = [216, 67, 82]

      #saving mask images to a new folder
      save_img('Development_Dataset/Test_Dev/Bipbip/Haricot/Masks/' + mask_img + '.png',prediction_img)

      mask_arr=read_rgb_mask("Development_Dataset/Test_Dev/Bipbip/Haricot/Masks/" + mask_img + '.png')

      submission_dict[mask_img] = {}
      submission_dict[mask_img]['shape'] = mask_arr.shape
      submission_dict[mask_img]['team'] = team
      submission_dict[mask_img]['crop'] = crop
      submission_dict[mask_img]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[mask_img]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[mask_img]['segmentation']['weed'] = rle_encoded_weed
      print(submission_dict)
    return submission_dict


In [ ]:
#reading rgb masks

import os
import numpy as np
from PIL import Image


def read_rgb_mask(img_path):
    '''
    img_path: path to the mask file
    Returns the numpy array containing target values
    '''

    mask_img = Image.open(img_path)
    mask_arr = np.array(mask_img)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2

    return new_mask_arr


In [ ]:
import os
import json
import numpy as np
from PIL import Image


def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
#def submission():
submission_dict = {}

submission_dict = submit(submission_dict,'Bipbip','Haricot')

In [ ]:
with open('submission.json', 'w') as f:
  json.dump(submission_dict, f)